In [68]:
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import pandas as pd
from random import random
import json, os.path, pickle

#result = pd.read_csv(f"test10.csv")
result = pd.read_csv(f"test10a.2.csv")
result['expected_n_attempts'] = result['students_per_group']*result['attempts_per_student']*result['trials']*2
result.columns

Index(['Unnamed: 0', 'student_set', 'comparision', 'students_per_group',
       'attempts_per_student', 'groupA', 'groupB', 'total_attempts', 'trials',
       'geeBIswrong', 'geeBIsns', 'geeBIssig', 'geeBIsvsig', 'geeBIsp',
       'query', 'time', 'key', 'expected_n_attempts'],
      dtype='object')

In [72]:
fields = ['student_set', 'comparision', 'students_per_group',
       'attempts_per_student',
        'total_attempts', 'trials', 'expected_n_attempts',
          'groupA', 'groupB']
query = "expected_n_attempts != total_attempts"
bad = result.query(query)[fields]
if len(bad):
    print('size errors!')
    display(bad)
else:
    print('no size errors')

In [75]:
result.comparision.distinct()

AttributeError: 'Series' object has no attribute 'distinct'

In [74]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"`{c}` < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
query = "comparision == 'small' and students_per_group in (75, 100)"
query = "comparision == 'mod' and students_per_group == 100"
print(query)
fields = ['student_set', 'comparision', 'students_per_group', 
       'attempts_per_student', 'trials',
          #'groupA', 'groupB', 'total_attempts', 'trials',
       #'geeBwrong', 'geeBns', 'geeBsig', 'geeBvsig', 'geeBp', 
       #'geeBIwrong', 'geeBIns', 'geeBIsig', 'geeBIvsig', 'geeBIp', 
       'geeBIswrong', 'geeBIsns', 'geeBIssig', 'geeBIsvsig', 'geeBIsp'
         ]
result.query(query)[fields]

Finding powered combinations where count of NS trials is less than 80%
comparision == 'mod' and students_per_group == 100


,student_set,comparision,students_per_group,attempts_per_student,trials,geeBIswrong,geeBIsns,geeBIssig,geeBIsvsig,geeBIsp
50,variable,mod,100,10,100,0,2,8,90,5.022250e-01
51,variable,mod,100,20,100,0,1,8,91,3.433378e-01
148,uniform,mod,100,10,100,0,0,0,100,1.345664e-03
149,uniform,mod,100,20,100,0,0,0,100,1.271928e-07
246,initial,mod,100,10,100,0,0,0,100,2.315328e-03
247,initial,mod,100,20,100,0,0,0,100,3.258328e-04
344,second,mod,100,10,100,0,12,22,66,2.574211e+00
345,second,mod,100,20,100,0,4,13,83,8.164139e-01


In [44]:
# Look for invalid measures:
print("These counts chould be ~2.5")
result.query("comparision == 'none'")[[c for c in result.columns if 'wrong' in c]]


These counts chould be ~2.5


,geeBIswrong
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [46]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"{c} < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
result.query(query)

Finding powered combinations where count of NS trials is less than 80%


,Unnamed: 0,student_set,comparision,students_per_group,attempts_per_student,groupA,groupB,total_attempts,trials,geeBIswrong,geeBIsns,geeBIssig,geeBIsvsig,geeBIsp,query,time,key
14,14,variable,vsmall,20,10,control,interventionA,1000,10,0,9,1,0,3.648460,"group == ['control', 'interventionA'] and pint...",1.626863e+12,d.10vsmall.20.10
15,15,variable,vsmall,20,20,control,interventionA,1000,10,0,9,1,0,3.648460,"group == ['control', 'interventionA'] and pint...",1.626863e+12,d.10vsmall.20.20
16,16,variable,vsmall,30,10,control,interventionA,1000,10,0,9,1,0,3.648460,"group == ['control', 'interventionA'] and pint...",1.626863e+12,d.10vsmall.30.10
17,17,variable,vsmall,30,20,control,interventionA,1000,10,0,9,1,0,3.648460,"group == ['control', 'interventionA'] and pint...",1.626863e+12,d.10vsmall.30.20
18,18,variable,vsmall,50,10,control,interventionA,1000,10,0,9,1,0,3.648460,"group == ['control', 'interventionA'] and pint...",1.626863e+12,d.10vsmall.50.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,93,second,upper,100,20,interventionC,interventionD,1000,10,1,9,0,0,3.069262,"group == ['interventionC', 'interventionD'] an...",1.626863e+12,d.10upper.100.20
388,94,second,upper,200,10,interventionC,interventionD,1000,10,1,9,0,0,3.069262,"group == ['interventionC', 'interventionD'] an...",1.626863e+12,d.10upper.200.10
389,95,second,upper,200,20,interventionC,interventionD,1000,10,1,9,0,0,3.069262,"group == ['interventionC', 'interventionD'] an...",1.626863e+12,d.10upper.200.20
390,96,second,upper,400,10,interventionC,interventionD,1000,10,1,9,0,0,3.069262,"group == ['interventionC', 'interventionD'] an...",1.626863e+12,d.10upper.400.10
